In [1]:
import hyperspy.api as hs
import numpy as np
import plotly.graph_objects as go
import pandas as pd
import os

In [4]:
hs.material.elements.Ga.Atomic_properties.Xray_lines.Ka['energy (keV)']

9.2517

In [5]:
##### SEM Apreo March 2023 #####
path = '../../Masteroppgave/2023-03-08_EDS-Apreo/exports/'

In [16]:
aztec_TEM_file = '../../Masteroppgave/2023-03-08_EDS-Apreo/AZtec quant as EDS_TEM.xlsx'
df = pd.read_excel(aztec_TEM_file, sheet_name=1)

In [17]:
df.head(2)

,File,Group,Element,Line series,k-factor,Line,AZ %at,HS %at s,HS %at s A,HS %at m,HS %at m A t=5e+01,HS %at m A t=1e+02,HS %at m A t=2e+02,HS %at m A t=4e+02,HS %at m A t=1e+03,HS %at m A t=1e+04,HS %at m A t=1e+05
0,GaAs_05kV_25pA,A,As,L,1.210,As_La,45.4,0,0,0,0,0,0,0,0,0,0
1,GaAs_05kV_25pA,A,Ga,L,1.086,Ga_La,54.6,0,0,0,0,0,0,0,0,0,0


In [18]:
def quant_as_TEM(filename, a_corr=True, noise_slice=0.65, print_results=False):

    # thickness_list = [1e2, 1e3,1e4,1e5] # in nm
    thickness_list = [50, 100, 200, 400, 1e3, 1e4, 1e5] # in nm


    # check if filename exists in path
    if filename + '.emsa' not in os.listdir(path):
        print(filename + '.emsa', 'not found in path')
        return
    
    df_s = df[df['File'] == filename]
    df_s = df_s.sort_values(by=['Element']) # alphabetically, as HS

    # make the spectrum and the model
    s = hs.load(path + filename + '.emsa', signal_type='EDS_TEM')
    elements = df_s['Element'].values
    s = s.isig[noise_slice:] # noise peak slice
    # if Vacc is lower than 20, slice at Vacc
    if s.metadata.Acquisition_instrument.TEM.beam_energy < 20:
        s = s.isig[:s.metadata.Acquisition_instrument.TEM.beam_energy]

    s.add_elements(elements)
    s.add_lines()
    m = s.create_model()
    m.fit()
    m.fit_background()

    lines = df_s['Line'].values
    kfactors = df_s['k-factor'].values
    intensities = s.get_lines_intensity()
    m_intensities = m.get_lines_intensity()

    # select only the relevant lines in m (relevant = we have a k-factor for it from AZtec)    
    lines_in_m_to_use = []
    for m_line in m_intensities:
        if m_line.metadata.Sample.xray_lines[0] in lines:
            lines_in_m_to_use.append(m_line)
    if a_corr:
    # calculate %at for each line in m, if possible
        for l in lines:
            try:
                m[l]
                for thickness in thickness_list:
                    at_m = s.quantification(lines_in_m_to_use, factors=kfactors, method='CL', absorption_correction=True, thickness=thickness)
                    df_s[f'HS %at m A t={thickness:.0e}'] = [at_m[0][0].data.round(2)[0], at_m[0][1].data.round(2)[0]]
            except:
                print(l, 'not found in m, quantification with model not possible')
                df_s[f'HS %at m A t={thickness:.0e}'] = [np.nan, np.nan]

        # do the CL quantification with the intensities from the original spectrum
        at = s.quantification(intensities, factors=kfactors, method='CL', absorption_correction=True, thickness=thickness)
        df_s['HS %at s A'] = [at[0][0].data.round(2)[0], at[0][1].data.round(2)[0]]
    
    else: # no absorption correction
        # almost the same code, but slicing at and at_m with one less level
        for l in lines:
            try:
                m[l]
                at_m = s.quantification(lines_in_m_to_use, factors=kfactors, method='CL')
                df_s['HS %at m'] = [at_m[0].data.round(2)[0], at_m[1].data.round(2)[0]]
            except:
                print(l, 'not found in m, quantification with model not possible')
                df_s['HS %at m'] = [np.nan, np.nan]
        at = s.quantification(intensities, factors=kfactors, method='CL')
        df_s['HS %at s'] = [at[0].data.round(2)[0], at[1].data.round(2)[0]]
            

    if print_results:
        print(df_s)

    # put results back in df
    df[df['File'] == filename] = df_s

In [20]:
quant_as_TEM('GaAs_30kV_25pA',print_results=True)

Absorption correction calculation: 0it [00:00, ?it/s]

[########################################] | 100% Completed | 300.44 ms


Absorption correction calculation: 0it [00:00, ?it/s]

[########################################] | 100% Completed | 114.34 ms


Absorption correction calculation: 0it [00:00, ?it/s]

...

[########################################] | 100% Completed | 121.40 ms


Absorption correction calculation: 0it [00:00, ?it/s]

[########################################] | 100% Completed | 114.54 ms


Absorption correction calculation: 0it [00:00, ?it/s]

[########################################] | 100% Completed | 121.48 ms
             File Group Element Line series  k-factor   Line  AZ %at  \
6  GaAs_30kV_25pA     A      As           K     4.191  As_Ka   42.03   
7  GaAs_30kV_25pA     A      Ga           K     3.268  Ga_Ka   57.97   

   HS %at s  HS %at s A  HS %at m  HS %at m A t=5e+01  HS %at m A t=1e+02  \
6         0       62.03         0               41.58               41.63   
7         0       37.97         0               58.42               58.37   

   HS %at m A t=2e+02  HS %at m A t=4e+02  HS %at m A t=1e+03  \
6               41.75               41.97               42.63   
7               58.25               58.03               57.37   

   HS %at m A t=1e+04  HS %at m A t=1e+05  
6               49.93               61.36  
7               50.07               38.64  


In [21]:
quant_as_TEM('GaAs_30kV_25pA',print_results=True, a_corr=False)


[########################################] | 100% Completed | 110.14 ms
[########################################] | 100% Completed | 106.92 ms
[########################################] | 100% Completed | 105.15 ms
             File Group Element Line series  k-factor   Line  AZ %at  \
6  GaAs_30kV_25pA     A      As           K     4.191  As_Ka   42.03   
7  GaAs_30kV_25pA     A      Ga           K     3.268  Ga_Ka   57.97   

   HS %at s  HS %at s A  HS %at m  HS %at m A t=5e+01  HS %at m A t=1e+02  \
6     42.54       62.03     41.52               41.58               41.63   
7     57.46       37.97     58.48               58.42               58.37   

   HS %at m A t=2e+02  HS %at m A t=4e+02  HS %at m A t=1e+03  \
6               41.75               41.97               42.63   
7               58.25               58.03               57.37   

   HS %at m A t=1e+04  HS %at m A t=1e+05  
6               49.93               61.36  
7               50.07               38.64  


In [ ]:
quant_as_TEM('GaAs_30kV_25pA',print_results=True)
quant_as_TEM('GaAs_30kV_25pA',print_results=True, a_corr=False)



In [22]:
# took 3-4 min
for f in df['File'].unique():
    print(f)
    quant_as_TEM(f)

GaAs_05kV_25pA


Absorption correction calculation: 0it [00:00, ?it/s]

[########################################] | 100% Completed | 111.22 ms


Absorption correction calculation: 0it [00:00, ?it/s]

[########################################] | 100% Completed | 109.78 ms


...

[########################################] | 100% Completed | 115.22 ms


Absorption correction calculation: 0it [00:00, ?it/s]

[########################################] | 100% Completed | 106.15 ms


Absorption correction calculation: 0it [00:00, ?it/s]

[########################################] | 100% Completed | 108.85 ms


In [23]:
# took 30 sec
for f in df['File'].unique():
    print(f)
    quant_as_TEM(f, a_corr=False)

GaAs_05kV_25pA
[########################################] | 100% Completed | 117.81 ms
[########################################] | 100% Completed | 105.61 ms
[########################################] | 100% Completed | 113.05 ms
GaAs_10kV_25pA
[########################################] | 100% Completed | 115.12 ms
[########################################] | 100% Completed | 105.22 ms
[########################################] | 100% Completed | 107.00 ms
GaAs_15kV_25pA
[########################################] | 100% Completed | 114.19 ms
[########################################] | 100% Completed | 119.00 ms
[########################################] | 100% Completed | 119.15 ms
GaAs_30kV_25pA
[########################################] | 100% Completed | 114.66 ms
[########################################] | 100% Completed | 118.20 ms
[########################################] | 100% Completed | 121.87 ms
GaAs_30kV_50pA
[########################################] | 100% Completed |

In [24]:
df.head(3)

,File,Group,Element,Line series,k-factor,Line,AZ %at,HS %at s,HS %at s A,HS %at m,HS %at m A t=5e+01,HS %at m A t=1e+02,HS %at m A t=2e+02,HS %at m A t=4e+02,HS %at m A t=1e+03,HS %at m A t=1e+04,HS %at m A t=1e+05
0,GaAs_05kV_25pA,A,As,L,1.210,As_La,45.40,45.00,62.93,45.41,46.90,48.24,50.56,53.98,59.51,63.19,63.19
1,GaAs_05kV_25pA,A,Ga,L,1.086,Ga_La,54.60,55.00,37.07,54.59,53.10,51.76,49.44,46.02,40.49,36.81,36.81
2,GaAs_10kV_25pA,A,As,L,1.310,As_La,42.09,41.43,60.58,41.62,43.22,44.65,47.15,50.84,56.79,60.71,60.71


In [ ]:
y_keys = ['AZ %at', 'HS %at s','HS %at m A t=5e+01', 'HS %at m A t=1e+02', 'HS %at m A t=2e+02', 'HS %at m A t=4e+02', 'HS %at m A t=1e+03', 'HS %at m A t=1e+04', 'HS %at m A t=1e+05']
x_key = ['File']



In [34]:
hs.material.elements[['Ga']]['General_properties']['Z']

31

In [36]:
hs.material.elements.Cu.General_properties

├── Z = 29
├── atomic_weight = 63.546
└── name = copper

In [ ]:
yolo = 

In [26]:
# save the df to excel
df.to_excel('results/quant_as_TEM.xlsx')

In [ ]:
fig = go.Figure()
lines_list = ['Sb_La', 'As_Ka', 'Ga_Ka', 'Ga_La', 'As_La']
colors = ['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3', '#FF6692', '#B6E880', '#FF97FF', '#FECB52'] # plotly colors

for l in lines_list:
    for y in y_keys:
        i = y_keys.index(y)
        fig.add_scatter(x=df[df['Line'] == l]['File'], y = df[df['Line'] == l][y], mode='markers', name=y, marker=dict(size=8, color=colors[i]), legendgroup=y, showlegend=True if l == lines_list[0] else False)
fig.update_layout(title='Data treated as TEM, quantified with HS using different quantification parameters<br><sub>s=original spectrum, m=model, A=absorption correction, t=thickness')
fig.update_layout(legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1))
fig.update_yaxes(range=[0, 100])
fig

In [ ]:
# fig.write_html('results/quant_as_TEM.html')